# Data Integration from Multiple Sources

This notebook demonstrates how to integrate data from different sources using schema alignment and merging techniques. We'll work with data from CSV and XML files, showing how to handle different formats and schemas.

**Learning Objectives:**
- Understand data integration challenges
- Learn to parse different data formats (CSV, XML)
- Master schema alignment techniques
- Implement intelligent data merging strategies

**Prerequisites:**
- Basic Python programming
- Familiarity with pandas and data structures
- Understanding of XML structure
- Knowledge of data merging concepts

In [1]:
# Import the required libraries
import xml.etree.ElementTree as ET
import pandas as pd

## Step 1: Load CSV Data (Sales Database)

We'll start by loading customer sales data from a CSV file.

In [2]:
# Reading the CSV file (Sales Database)
df_sales = pd.read_csv("./data/customer_sales.csv")

print("Original Customer Sales Data:")
print("=" * 92)
print(df_sales.head())
print("=" * 92)
print(df_sales.tail())
print("=" * 92)

Original Customer Sales Data:
   customer_id        name  sales_amount  sales_date
0            1  Customer 1           202  2023-01-01
1            2  Customer 2           535  2023-01-02
2            3  Customer 3           960  2023-01-03
3            4  Customer 4           370  2023-01-04
4            5  Customer 5           206  2023-01-05
    customer_id          name  sales_amount  sales_date
95           96   Customer 96           301  2023-04-06
96           97   Customer 97           369  2023-04-07
97           98   Customer 98           962  2023-04-08
98           99   Customer 99           915  2023-04-09
99          100  Customer 100           370  2023-04-10


## Step 2: Define XML Parser Function

We need a custom function to parse XML data and convert it to a pandas DataFrame.

In [3]:
def parse_xml(file: str) -> pd.DataFrame:
    """Parse an XML file and return a DataFrame with the data.
    
    Args:
        file (str): The path to the XML file.

    Returns:
        pd.DataFrame: A DataFrame containing the data from the XML file.
    """

    # Parse the XML file
    tree = ET.parse(file)
    root = tree.getroot()

    # Extract the data from the XML file
    data = []
    for customer in root.findall("customer"):
        id_ = int(customer.find("id").text)
        name = customer.find("full_name").text
        email = customer.find("email").text
        purchase = customer.find("purchase").text
        signup_date = customer.find("signup_date").text
        data.append([id_, name, email, purchase, signup_date])

    # Creating a DataFrame
    df = pd.DataFrame(data, columns=[
        "customer_id", "name", "email", "sales_amount", "signup_date"
    ])

    return df

## Step 3: Load XML Data (Marketing Database)

Now we'll use our parser function to load customer marketing data from an XML file.

In [4]:
# Reading the XML file (Marketing Database)
df_marketing = parse_xml("./data/customer_marketing.xml")

print("\nOriginal Customer Marketing Data:")
print("=" * 92)
print(df_marketing.head())
print("=" * 92)
print(df_marketing.tail())
print("=" * 92)


Original Customer Marketing Data:
   customer_id         name                   email sales_amount signup_date
0            2   Customer 2   customer2@example.com          535  2023-01-02
1            6   Customer 6   customer6@example.com          171  2023-01-06
2           10  Customer 10  customer10@example.com          221  2023-01-10
3           16  Customer 16  customer16@example.com          472  2023-01-16
4           17  Customer 17  customer17@example.com          199  2023-01-17
    customer_id          name                    email sales_amount  \
95          171  Customer 171  customer171@example.com          851   
96          172  Customer 172  customer172@example.com          243   
97          173  Customer 173  customer173@example.com          708   
98          174  Customer 174  customer174@example.com          300   
99          175  Customer 175  customer175@example.com          223   

   signup_date  
95  2023-04-12  
96  2023-04-13  
97  2023-04-14  
98  2023

## Step 4: Schema Alignment

Before merging, we need to align the schemas by converting columns to appropriate data types.

In [5]:
# Schema Alignment: Convert columns to appropriate types
df_marketing["sales_amount"] = pd.to_numeric(
    df_marketing["sales_amount"], errors="coerce").fillna(0)
df_sales["sales_date"] = pd.to_datetime(df_sales["sales_date"])
df_marketing["signup_date"] = pd.to_datetime(df_marketing["signup_date"])

print("Data types after schema alignment:")
print("\nSales DataFrame dtypes:")
print(df_sales.dtypes)
print("\nMarketing DataFrame dtypes:")
print(df_marketing.dtypes)

Data types after schema alignment:

Sales DataFrame dtypes:
customer_id              int64
name                    object
sales_amount             int64
sales_date      datetime64[ns]
dtype: object

Marketing DataFrame dtypes:
customer_id              int64
name                    object
email                   object
sales_amount             int64
signup_date     datetime64[ns]
dtype: object


## Step 5: Intelligent Data Merging

We'll merge the data from both sources, handling duplicate columns intelligently by prioritizing certain data sources.

In [6]:
# Merging the CSV and XML data intelligently
# Merge on "customer_id", prioritize "df_sales" data when duplicate columns exist
df_merged = pd.merge(df_sales, df_marketing, on="customer_id", how="outer",
                     suffixes=("_sales", "_marketing"))

print("Initial merged DataFrame shape:", df_merged.shape)
print("Columns after initial merge:", df_merged.columns.tolist())

Initial merged DataFrame shape: (175, 8)
Columns after initial merge: ['customer_id', 'name_sales', 'sales_amount_sales', 'sales_date', 'name_marketing', 'email', 'sales_amount_marketing', 'signup_date']


## Step 6: Resolve Duplicate Columns

We'll resolve duplicate columns by prioritizing non-null values from the sales data and filling missing values with marketing data.

In [7]:
# Resolve the duplicated columns by:
#  - Prioritizing non-null values from the sales data
#  - Filling missing values with data from the marketing dataset
df_merged["name"] = df_merged["name_sales"].combine_first(
    df_merged["name_marketing"]
)
df_merged["sales_amount"] = df_merged["sales_amount_sales"].combine_first(
    df_merged["sales_amount_marketing"]
)
df_merged["sales_date"] = df_merged["sales_date"].combine_first(
    df_merged["signup_date"]
)

print("Resolved duplicate columns successfully")

Resolved duplicate columns successfully


## Step 7: Clean Up and Reorder Columns

We'll remove the temporary columns with suffixes and reorder the final DataFrame.

In [8]:
# Dropping the unnecessary columns (those with suffixes)
df_merged = df_merged.drop(columns=[
    "name_sales",
    "name_marketing",
    "sales_amount_sales",
    "sales_amount_marketing",
    "signup_date"
])

# Reorder the columns
df_merged = df_merged.reindex(columns=[
    "customer_id",
    "name",
    "email",
    "sales_amount",
    "sales_date"
])

print("Final DataFrame shape:", df_merged.shape)
print("Final columns:", df_merged.columns.tolist())

Final DataFrame shape: (175, 5)
Final columns: ['customer_id', 'name', 'email', 'sales_amount', 'sales_date']


## Step 8: Display the Unified Schema

Let's examine the final integrated dataset.

In [9]:
# Display the unified schema
print("\nUnified Data after Schema Integration:")
print("=" * 92)
print(df_merged.head())
print("=" * 92)
print(df_merged.tail())
print("=" * 92)

print(f"\nData Integration Summary:")
print(f"- Original Sales records: {len(df_sales)}")
print(f"- Original Marketing records: {len(df_marketing)}")
print(f"- Integrated records: {len(df_merged)}")
print(f"- Data types: {df_merged.dtypes.to_dict()}")


Unified Data after Schema Integration:
   customer_id        name                  email  sales_amount sales_date
0            1  Customer 1                    NaN         202.0 2023-01-01
1            2  Customer 2  customer2@example.com         535.0 2023-01-02
2            3  Customer 3                    NaN         960.0 2023-01-03
3            4  Customer 4                    NaN         370.0 2023-01-04
4            5  Customer 5                    NaN         206.0 2023-01-05
     customer_id          name                    email  sales_amount  \
170          171  Customer 171  customer171@example.com         851.0   
171          172  Customer 172  customer172@example.com         243.0   
172          173  Customer 173  customer173@example.com         708.0   
173          174  Customer 174  customer174@example.com         300.0   
174          175  Customer 175  customer175@example.com         223.0   

    sales_date  
170 2023-04-12  
171 2023-04-13  
172 2023-04-14  
173

## Summary

In this notebook, we successfully demonstrated data integration from multiple sources:

### Key Steps Accomplished:

1. **Data Loading**: Loaded data from CSV and XML files using appropriate parsers
2. **Schema Alignment**: Converted data types to ensure compatibility
3. **Intelligent Merging**: Combined datasets using outer join to preserve all records
4. **Conflict Resolution**: Prioritized data from sales database while filling gaps with marketing data
5. **Schema Unification**: Created a clean, unified schema with consistent column names

### Challenges Addressed:

- **Format Diversity**: Handled different file formats (CSV vs XML)
- **Schema Differences**: Aligned column names and data types
- **Data Conflicts**: Resolved duplicate information intelligently
- **Missing Values**: Filled gaps using data from multiple sources

### Best Practices Demonstrated:

- **Prioritization Strategy**: Sales data took precedence over marketing data
- **Data Validation**: Used error handling for numeric conversions
- **Clean Organization**: Removed temporary columns and reordered for clarity

This approach ensures data quality while maximizing information retention from all available sources.